### Importing necessary modules


In [1]:
import time
import matplotlib.pyplot as plt
from diffusion import (
    SequentialDiffusionEquation,
    OMPdiffusionEquation,
    CUDADiffusionEquation,
    BaseDiffusionEquation,
)
import pandas as pd
import numpy as np

### Utils Methods


In [2]:
def standard_deviation(arr: list) -> float:
    mean = sum(arr) / len(arr)
    return (sum((x - mean) ** 2 for x in arr) / len(arr)) ** 0.5

In [3]:
def measure_execution_time(
    Solver: BaseDiffusionEquation,
    N: int,
    total_eval: int,
    steps: int,
    n_threads: int = None,
) -> list:
    time_list = []
    for _ in range(total_eval):
        with Solver("../build/libDiffusionEquation.so", N=N) as solver:
            if n_threads:
                solver.set_num_threads(n_threads)

            start = time.time()
            for _ in range(steps):
                solver.step()
            end = time.time()

            time_list.append(end - start)
    return time_list

### Measure Execution time of the Different Solutions


#### Sequential implementation


In [ ]:
times = measure_execution_time(SequentialDiffusionEquation, 500, 10, 1000)
print("Time elapsed: ", sum(times) / len(times), "+/-", standard_deviation(times))

#### OpenMP implementation


In [ ]:
times = measure_execution_time(OMPdiffusionEquation, 500, 10, 1000)
print("Time elapsed: ", sum(times) / len(times), "+/-", standard_deviation(times))

#### CUDA implementation


In [ ]:
times = measure_execution_time(CUDADiffusionEquation, 500, 10, 1000)
print("Time elapsed: ", sum(times) / len(times), "+/-", standard_deviation(times))

### Firsts Results


Now that we can measure the execution time of both implementations, let's compare them and check if the OpenMP implementation is faster than the Sequential implementation


In [5]:
# Configs
N = 2000
total_evaluations = 15
total_steps = 500
num_threads_omp = [2, 4, 8, 16, 32]

In [6]:
# Just to make the notebook screen goes off before the execution
time.sleep(70)

In [7]:
seq_times = measure_execution_time(
    SequentialDiffusionEquation, N, total_evaluations, total_steps
)
seq_time_mean = sum(seq_times) / len(seq_times)
seq_std = standard_deviation(seq_times)

In [ ]:
omp_times_list = []
omp_std_list = []
for num_threads in num_threads_omp:
    times = measure_execution_time(
        OMPdiffusionEquation, N, total_evaluations, total_steps, num_threads
    )
    omp_times_list.append(sum(times) / len(times))
    omp_std_list.append(standard_deviation(times))

In [9]:
cuda_times = measure_execution_time(
    CUDADiffusionEquation, N, total_evaluations, total_steps
)
cuda_time_mean = sum(cuda_times) / len(cuda_times)
cuda_std = standard_deviation(cuda_times)

In [ ]:
table = {
    "Num Threads": [1] + num_threads_omp + ["CUDA"],
    "Time": [seq_time_mean] + omp_times_list + [cuda_time_mean],
    "STD": [seq_std] + omp_std_list + [cuda_std],
    "Speedup": [seq_time_mean / x for x in [seq_time_mean] + omp_times_list]
    + [seq_time_mean / cuda_time_mean],
    "Efficiency": [1] + [
        seq_time_mean / x / num_threads
        for x, num_threads in zip(omp_times_list, num_threads_omp)
    ]
    + [seq_time_mean / cuda_time_mean / 32],
}
print(table)
df = pd.DataFrame(table)
print(df)

path_to_save = "../data/performance/OMP_CUDA.csv"
df.to_csv(path_to_save, index=False)

#### Plot the results


In [ ]:
# plot the results together in a single graph
df = pd.read_csv(path_to_save)

plt.plot(df["Num Threads"], df["Speedup"], label="Speedup", marker="o")
# Add the linear speedup line
plt.plot(
    df["Num Threads"],
    df["Num Threads"],
    label="Speedup Linear",
    linestyle="--",
    marker="o",
)

# # Add value labels next to each data point
# for x, y in zip(df["Num Threads"], df["Speedup"]):
#     plt.text(x, y, f'{y:.2f}', fontsize=9, ha='right', va='bottom')

plt.ylim(1, df["Speedup"].max() + 0.5)  # Adjust the y-axis limit
plt.grid()
plt.title("Speedup vs Nº Threads")
plt.ylabel("Speedup")
plt.xlabel("Nº Threads")
plt.legend()
plt.show()

In [ ]:
# plot the results together in a single graph
df = pd.read_csv(path_to_save)

# Plot the efficiency
plt.plot(df["Num Threads"], df["Efficiency"], label="Eficiência", marker="o")
# Add the linear efficiency line
plt.plot(
    df["Num Threads"],
    [1] * len(df["Num Threads"]),
    label="Eficiência Linear",
    linestyle="--",
    marker="o",
)

plt.grid()
plt.title("Eficiência vs Nº Threads")
plt.xlabel("Nº Threads")
plt.ylabel("Eficiência")
plt.legend()
plt.show()

In [ ]:
df = pd.read_csv(path_to_save)

# Calculate percentage of linear speedup achieved
df["Percent of Linear Speedup"] = (df["Speedup"] / df["Num Threads"]) * 100

# Plot the percentage
plt.plot(
    df["Num Threads"],
    df["Percent of Linear Speedup"],
    label="Percent of Linear Speedup",
    marker="o",
)

plt.grid()
plt.title("Percentage of Linear Speedup Achieved vs Number of Threads")
plt.xlabel("Number of Threads")
plt.ylabel("Percentage of Linear Speedup Achieved (%)")
plt.legend()
plt.show()

In [ ]:
df = pd.read_csv(path_to_save)

fig, ax1 = plt.subplots()

color = "tab:blue"
ax1.set_xlabel("Number of Threads")
ax1.set_ylabel("Measured Speedup", color=color)
ax1.plot(
    df["Num Threads"], df["Speedup"], label="Measured Speedup", color=color, marker="o"
)
ax1.tick_params(axis="y", labelcolor=color)

ax2 = ax1.twinx()  # Instantiate a second axes sharing the same x-axis

color = "tab:red"
ax2.set_ylabel("Linear Speedup", color=color)
ax2.plot(
    df["Num Threads"],
    df["Num Threads"],
    label="Linear Speedup",
    linestyle="--",
    color=color,
)
ax2.tick_params(axis="y", labelcolor=color)

# Combined legend
lines_labels = [ax.get_legend_handles_labels() for ax in [ax1, ax2]]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
fig.legend(lines, labels, loc="upper left")

plt.title("Speedup vs Number of Threads")
plt.grid()
plt.show()